# Iteration 5: Human-in-the-Loop (Simplified)

This notebook demonstrates human-in-the-loop using LangGraph's built-in interrupt functionality.
This is a much simpler and more production-ready approach than custom implementations.

## The Concept: LangGraph Interrupts

LangGraph provides a built-in way to pause graph execution and wait for human input:

1. **Checkpointing** - Graph state is saved when interrupted
2. **interrupt_before** - Specify nodes that need human approval
3. **Resume** - Continue execution with human-provided updates

This approach is:
- ✅ Simple to implement
- ✅ Production-ready
- ✅ Handles async naturally
- ✅ Maintains conversation context

In [ ]:
import sys
sys.path.append('..')

from modules.m5_human_in_loop_simple import (
    HumanInLoopState,
    check_claim_with_human_review,
    resume_after_human_input,
    create_human_in_loop_graph
)
from langgraph.graph import StateGraph
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports successful!")

## 1. Understanding the Flow

Our simplified human-in-the-loop flow:

1. AI analyzes the claim
2. If confidence < 50% or verdict is UNCERTAIN → Request human review
3. Graph **interrupts** before human_review node
4. Human provides feedback
5. Graph **resumes** with human input

In [ ]:
# Visualize the graph structure
import base64
from IPython.display import Image

app = create_human_in_loop_graph()
mermaid_code = app.get_graph().draw_mermaid()

# Render the graph
graph_bytes = mermaid_code.encode("utf-8")
base64_string = base64.b64encode(graph_bytes).decode("ascii")
image_url = f"https://mermaid.ink/img/{base64_string}?type=png"

print("📊 Human-in-the-Loop Graph Structure:")
Image(url=image_url)

## 2. Testing Different Scenarios

In [ ]:
# Test claims with different confidence levels
test_scenarios = [
    {
        "claim": "The Boeing 747 has four engines",
        "expected": "High confidence, no review needed"
    },
    {
        "claim": "A startup just invented teleportation technology",
        "expected": "Low confidence, should trigger review"
    },
    {
        "claim": "SpaceX announced a Mars mission for next week",
        "expected": "Current event with uncertainty"
    }
]

print("🧪 Testing Human-in-the-Loop Scenarios\n")

for i, scenario in enumerate(test_scenarios):
    thread_id = f"test_{i}"
    print(f"\n{'='*70}")
    print(f"Scenario {i+1}: {scenario['expected']}")
    print(f"Claim: \"{scenario['claim']}\"")
    print("="*70)
    
    # Run the claim
    result = check_claim_with_human_review(scenario['claim'], thread_id)
    
    if result:
        # No human review needed
        print(f"\n✅ AI Decision (No Review Needed):")
        print(f"Verdict: {result['verdict']} ({result['confidence']}%)")
        print(f"Reasoning: {result['reasoning'][:100]}...")
    else:
        # Human review needed - graph is interrupted
        print(f"\n🤚 GRAPH INTERRUPTED - Human Review Required")
        print("\nSimulating human input...")
        
        # Simulate different human responses
        if "teleportation" in scenario['claim']:
            human_verdict = "BS"
            human_confidence = 99
            human_reasoning = "Teleportation violates known physics"
        else:
            human_verdict = "UNCERTAIN"
            human_confidence = 60
            human_reasoning = "Need to verify with official sources"
        
        # Resume with human input
        final_result = resume_after_human_input(
            thread_id,
            human_verdict,
            human_confidence,
            human_reasoning
        )
        
        print(f"\n✅ Final Decision (After Human Review):")
        print(f"Verdict: {final_result['verdict']} ({final_result['confidence']}%)")
        print(f"Reasoning: {final_result['reasoning']}")
        print(f"Human Reviewed: {final_result.get('human_reviewed', False)}")

## 3. Interactive Human Review

Let's create an interactive example where you can provide the human review:

In [ ]:
# Interactive claim checking
def interactive_check(claim: str):
    """Interactive function for human review"""
    import uuid
    thread_id = str(uuid.uuid4())
    
    print(f"\n🔍 Checking claim: \"{claim}\"")
    print("="*60)
    
    result = check_claim_with_human_review(claim, thread_id)
    
    if result:
        print(f"\n✅ AI made a confident decision:")
        print(f"Verdict: {result['verdict']} ({result['confidence']}%)")
        print(f"Reasoning: {result['reasoning']}")
        return result
    else:
        print(f"\n🤚 Human review required!")
        print("\nPlease provide your assessment:")
        
        # Get human input
        verdict = input("Verdict (BS/LEGITIMATE/UNCERTAIN): ").upper()
        while verdict not in ["BS", "LEGITIMATE", "UNCERTAIN"]:
            verdict = input("Please enter BS, LEGITIMATE, or UNCERTAIN: ").upper()
        
        confidence = int(input("Confidence (0-100): "))
        reasoning = input("Reasoning: ")
        
        # Resume with human input
        final_result = resume_after_human_input(
            thread_id,
            verdict,
            confidence,
            reasoning
        )
        
        print(f"\n✅ Final result:")
        print(f"Verdict: {final_result['verdict']} ({final_result['confidence']}%)")
        print(f"Reasoning: {final_result['reasoning']}")
        
        return final_result

# Test with an ambiguous claim
test_claim = "A new AI system passed the Turing test yesterday"
# interactive_check(test_claim)  # Uncomment to run interactively

## 4. Production Patterns

In a production system, you would typically:

1. **Queue interrupted requests** for human review
2. **Send notifications** to reviewers
3. **Track thread IDs** to resume later
4. **Set timeouts** for human responses

In [ ]:
# Example production pattern
class ProductionHumanReview:
    """Production-ready human review system"""
    
    def __init__(self):
        self.pending_reviews = {}  # thread_id -> claim info
        self.completed_reviews = {}
    
    def process_claim(self, claim: str) -> dict:
        """Process a claim, queuing for human review if needed"""
        import uuid
        thread_id = str(uuid.uuid4())
        
        # First pass
        result = check_claim_with_human_review(claim, thread_id)
        
        if result:
            # AI was confident
            return {
                "status": "completed",
                "result": result,
                "thread_id": thread_id
            }
        else:
            # Queue for human review
            self.pending_reviews[thread_id] = {
                "claim": claim,
                "status": "pending_review",
                "created_at": datetime.now()
            }
            
            # In production: send to queue, notify reviewers, etc.
            return {
                "status": "pending_review",
                "thread_id": thread_id,
                "message": "Claim queued for human review"
            }
    
    def submit_review(self, thread_id: str, verdict: str, 
                     confidence: int, reasoning: str) -> dict:
        """Submit human review and get final result"""
        if thread_id not in self.pending_reviews:
            return {"error": "Unknown thread_id"}
        
        # Resume graph with human input
        result = resume_after_human_input(
            thread_id, verdict, confidence, reasoning
        )
        
        # Move to completed
        review_info = self.pending_reviews.pop(thread_id)
        review_info["completed_at"] = datetime.now()
        review_info["result"] = result
        self.completed_reviews[thread_id] = review_info
        
        return {
            "status": "completed",
            "result": result,
            "thread_id": thread_id
        }
    
    def get_pending_reviews(self):
        """Get all pending reviews"""
        return [
            {
                "thread_id": tid,
                "claim": info["claim"],
                "waiting_time": (datetime.now() - info["created_at"]).seconds
            }
            for tid, info in self.pending_reviews.items()
        ]

# Demo the production pattern
from datetime import datetime

system = ProductionHumanReview()

# Process some claims
claims = [
    "The Wright brothers flew in 1903",  # High confidence
    "Cold fusion was achieved last month",  # Needs review
]

results = []
for claim in claims:
    result = system.process_claim(claim)
    results.append(result)
    print(f"\nProcessed: {claim[:30]}...")
    print(f"Status: {result['status']}")

# Check pending reviews
pending = system.get_pending_reviews()
if pending:
    print(f"\n📋 Pending Reviews: {len(pending)}")
    for review in pending:
        print(f"  - {review['claim'][:50]}... (waiting {review['waiting_time']}s)")
        
        # Simulate human review
        print("\n  Simulating human review...")
        final = system.submit_review(
            review['thread_id'],
            "BS",
            95,
            "Cold fusion remains unproven"
        )
        print(f"  Final verdict: {final['result']['verdict']}")

## 5. Benefits of LangGraph's Approach

### ✅ Built-in Features:
- **Checkpointing** - Automatic state persistence
- **Thread Management** - Conversation context maintained
- **Async Support** - Non-blocking by design
- **Error Handling** - Graceful failure recovery

### ✅ Production Ready:
- **Scalable** - Can handle many concurrent reviews
- **Reliable** - State persisted across restarts
- **Flexible** - Easy to integrate with queues/databases
- **Observable** - Built-in monitoring capabilities

In [ ]:
# Example: Monitoring human review metrics
class ReviewMetrics:
    def __init__(self):
        self.total_claims = 0
        self.reviews_triggered = 0
        self.avg_confidence_before = 0
        self.avg_confidence_after = 0
        self.review_reasons = {}
    
    def track_claim(self, needs_review: bool, reason: str = None, 
                   ai_confidence: int = None, human_confidence: int = None):
        self.total_claims += 1
        
        if needs_review:
            self.reviews_triggered += 1
            if reason:
                self.review_reasons[reason] = self.review_reasons.get(reason, 0) + 1
            
            if ai_confidence and human_confidence:
                # Update running averages
                n = self.reviews_triggered
                self.avg_confidence_before = (
                    (self.avg_confidence_before * (n-1) + ai_confidence) / n
                )
                self.avg_confidence_after = (
                    (self.avg_confidence_after * (n-1) + human_confidence) / n
                )
    
    def get_summary(self):
        review_rate = self.reviews_triggered / self.total_claims if self.total_claims else 0
        
        return {
            "total_claims": self.total_claims,
            "review_rate": f"{review_rate:.1%}",
            "avg_confidence_improvement": 
                f"{self.avg_confidence_before:.0f}% → {self.avg_confidence_after:.0f}%",
            "top_reasons": sorted(
                self.review_reasons.items(), 
                key=lambda x: x[1], 
                reverse=True
            )[:3]
        }

# Demo metrics
metrics = ReviewMetrics()

# Simulate some tracking
metrics.track_claim(needs_review=False)
metrics.track_claim(needs_review=True, reason="Low confidence: 30%", 
                   ai_confidence=30, human_confidence=95)
metrics.track_claim(needs_review=True, reason="Current event uncertainty",
                   ai_confidence=45, human_confidence=80)
metrics.track_claim(needs_review=False)

print("📊 Human Review Metrics:")
summary = metrics.get_summary()
for key, value in summary.items():
    if key == "top_reasons":
        print(f"\nTop Review Reasons:")
        for reason, count in value:
            print(f"  - {reason}: {count}")
    else:
        print(f"{key}: {value}")

## Summary

### What We Built:
✅ **Simple Implementation** - Using LangGraph's built-in interrupts  
✅ **Production Ready** - Checkpointing, thread management included  
✅ **Flexible Integration** - Easy to add queues, notifications, etc.  
✅ **Maintains Context** - Full conversation history preserved  

### Key Advantages:
- 🎯 **Less Code** - LangGraph handles the complexity
- 🔄 **Reliable** - State persisted automatically
- ⚡ **Async Native** - Non-blocking by design
- 📊 **Observable** - Easy to add monitoring

### Next Steps:
In Iteration 6, we'll add **Memory/Persistence** to:
- Remember past decisions
- Learn from human feedback
- Build a knowledge base
- Reduce repeated reviews